## Producer Script for Live-Data from API-Call

#### Imports

In [47]:
import json
from kafka import KafkaProducer, KafkaAdminClient
from kafka.admin import NewTopic
import requests
import datetime

In [48]:
admin = KafkaAdminClient(bootstrap_servers="localhost:9092")
producer = KafkaProducer(bootstrap_servers="localhost:9092")

### Try to create topics

In [49]:
if "preise" not in admin.list_topics() and "stationen" not in admin.list_topics():
    new_topics= list()
    new_topics.append(NewTopic(name="preise", num_partitions=1, replication_factor=1))
    new_topics.append(NewTopic(name="stationen", num_partitions=1, replication_factor=1))
    admin.create_topics(new_topics)

In [50]:
def add_price(date, uuid, diesel, e5,e10):
    j = json.dumps({"date": date, "station_uuid":uuid,"diesel":diesel,"e10":e10,"e5":e5})
    print(j)
    producer.send("preise",str(j).encode())

def add_station( uuid, name, brand, street, house_number, post_code,city, lat, lng):
    j = json.dumps({"uuid":uuid,"name":name,"brand":brand,"street":street, "house_number":house_number,"post_code":post_code,"city":city,"latitude":lat,"longitude":lng,"first_active":"","openingtimes_json":None})
    print(j)
    producer.send("stationen",str(j).encode())

#### API Call

In [52]:
API_KEY = "e82265a8-b634-ed81-69d2-82b1249c39ea"
BASE_URL = "https://creativecommons.tankerkoenig.de/json/list.php?"
LAT=49.495411
LNG=8.461562
RAD=10.0
response = requests.get(BASE_URL+"lat="+str(LAT)+"&lng="+str(LNG)+"&rad="+str(RAD)+"&type=all&apikey="+API_KEY)
if response.status_code==200 and response.json()["ok"] is True:
    response = response.json()
    results = response["stations"]
    for r in results:
        add_price(date=datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S+01"), uuid=r["id"], diesel=r["diesel"], e5=r["e5"], e10=r["e10"])
        add_station(uuid=r["id"],name=r["name"],brand=r["brand"],street=r["street"], city=r["place"], house_number=r["houseNumber"], post_code=r["postCode"], lat=r["lat"],lng=r["lng"])

{"date": "2020-11-09 15:35:12+01", "station_uuid": "8c36f834-0060-4623-8213-6f6eb98cd69a", "diesel": 1.039, "e10": 1.159, "e5": 1.209}
{"uuid": "8c36f834-0060-4623-8213-6f6eb98cd69a", "name": "Aral Tankstelle", "brand": "ARAL", "street": "Hafenstra\u00dfe", "house_number": "19-21", "post_code": 68159, "city": "Mannheim", "latitude": 49.4940567, "longitude": 8.457063, "first_active": "", "openingtimes_json": null}
{"date": "2020-11-09 15:35:12+01", "station_uuid": "70543ec9-45a3-49e2-a86c-2d2fa7ece892", "diesel": 1.009, "e10": 1.119, "e5": 1.169}
{"uuid": "70543ec9-45a3-49e2-a86c-2d2fa7ece892", "name": "Sb Mannheim Waldhofstrasse. 62", "brand": "SB", "street": "Waldhofstrasse.", "house_number": "62", "post_code": 68169, "city": "Mannheim", "latitude": 49.502212, "longitude": 8.471017, "first_active": "", "openingtimes_json": null}
{"date": "2020-11-09 15:35:12+01", "station_uuid": "a674473d-bdc4-4e05-9677-e4dd74ed0184", "diesel": 1.019, "e10": 1.149, "e5": 1.199}
{"uuid": "a674473d-bdc4

Close connection to producer

In [22]:
producer.close()